# Latent Dirichlet Allocation

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions.
* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.

* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial.

* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution.


## Pipeline
    Data Preprocess(Tokenize --> Stopwords Removing --> Lemmatize --> stem)

                            |
                Generate BOW and Tfidf
                            
                            |
                   LDA on BOW and Tfidf
                            
                            |
                  performance evaluation
                        

# Step 1: Load Dataset

In [1]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

In [2]:
'''
Get the total number of documents
'''
print(len(documents))
print(documents[:5])

300000
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


# Step 2: Data Preprocessing

We will perform the following steps:

* Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
Eg. "John went to school"  --> ["John","went","to", "school"]

* All stopwords are removed.

* Words are lemmatized - words in third person are changed to first person and verbs in past and future tenses are changed into present. Eg. "went"  --> "go"

* Words are stemmed - words are reduced to their root form. Eg. "files"  --> "file"

In [4]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zhuangsheng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizer Example

* Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':



In [5]:
print(WordNetLemmatizer().lemmatize("went", pos = 'v'))  # past tense to present tense

go


### Stemmer Example
* Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [7]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']

singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data = {'original word':original_words,'stemmed':singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [10]:
gensim.utils.simple_preprocess("John went to office.")

['john', 'went', 'to', 'office']

In [16]:
'''
The function to perform the pre processing steps on the entire dataset
'''
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos = 'v'))

def preprocess(text):
    result = []
    # gensim.utils.simple_preprocess(text) --> tokenize
    for token in gensim.utils.simple_preprocess(text):
        # remove stopwords and words like "a","an","are"
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [17]:

'''
Preview a document after preprocessing
'''
# use one record as an example
document_num = 4310
doc_sample = documents[documents["index"] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(" "):
    words.append(word)
    
print(words)
print("\n\nTokenized and Lemmatized document:")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and Lemmatized document:
['rain', 'help', 'dampen', 'bushfir']


In [20]:
# Comapre
# Before processing the dataframe
print("before processing:")
print("\n")
print(documents.head())

before processing:


                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [21]:
print("after processing:")
print("\n")
processed_docs = documents["headline_text"].map(preprocess)
print(processed_docs[:6])

after processing:


0     [decid, communiti, broadcast, licenc]
1                        [wit, awar, defam]
2    [call, infrastructur, protect, summit]
3               [staff, aust, strike, rise]
4      [strike, affect, australian, travel]
5        [ambiti, olsson, win, tripl, jump]
Name: headline_text, dtype: object


# Step 3. Prepare data

## 3.1 Bag of Words
## 3.2 TF-IDF

## 3.1 Bag of Words

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass processed_docs to gensim.corpora.Dictionary() and call it 'dictionary'.

In [24]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''

dictionary = gensim.corpora.Dictionary(processed_docs)

In [25]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

12975 sari
11708 pullout
17469 mythsagain
18488 seashift
18191 calvin
8124 saga
23135 damon
19144 turndown
2524 hodg
12754 grapegrow
2624 room


#### Gensim filter_extremes

* filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [26]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''

dictionary.filter_extremes(no_below = 15, no_above = 0.1, keep_n = 100000)

#### Gensim doc2bow

doc2bow(document)

Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [27]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [28]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [29]:
'''
Preview BOW for our sample preprocessed document
'''

# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set


While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.

### TF-IDF stands for "Term Frequency, Inverse Document Frequency".

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document).
* IDF(w) = log_e(Total number of documents / Number of documents with term w in it).


### For example

* Consider a document containing 100 words wherein the word 'tiger' appears 3 times.
* The term frequency (i.e., tf) for 'tiger' is then:

    * TF = (3 / 100) = 0.03.
* Now, assume we have 10 million documents and the word 'tiger' appears in 1000 of these.Then, the inverse document frequency (i.e., idf) is calculated as:

    * IDF = log(10,000,000 / 1,000) = 4.
* Thus, the Tf-idf weight is the product of these quantities:

    * TF-IDF = 0.03 * 4 = 0.12.

In [30]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''


from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [31]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''

corpus_tfidf = tfidf[bow_corpus]

In [32]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


# LDA on Bag of Words and Tfidf

We are going for 10 topics in the document corpus.

We will be running LDA using all CPU cores to parallelize and speed up model training.

Some of the parameters we will be tweaking are:

* num_topics is the number of requested latent topics to be extracted from the training corpus.
* id2word is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* workers is the number of extra processes to use for parallelization. Uses all available cores by default.
* alpha and eta are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is 1/num_topics)

    * Alpha is the per document topic distribution.

        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics
    * Eta is the per topic word distribution.

        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.
* passes is the number of training passes through the corpus. For example, if the training corpus has 50,000 documents, chunksize is 10,000, passes is 2, then online training is done in 10 updates:

    * #1 documents 0-9,999
    * #2 documents 10,000-19,999
    * #3 documents 20,000-29,999
    * #4 documents 30,000-39,999
    * #5 documents 40,000-49,999
    * #6 documents 0-9,999
    * #7 documents 10,000-19,999
    * #8 documents 20,000-29,999
    * #9 documents 30,000-39,999
    * #10 documents 40,000-49,999

## Step 4.1: Running LDA using Bag of Words¶

In [34]:
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''

lda_model = gensim.models.LdaMulticore(
                                        bow_corpus,
                                        num_topics = 10,
                                        id2word = dictionary,
                                        passes = 2,
                                        workers = 2
                                        )

In [35]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx,topic))
    print("\n")

Topic: 0 
Words: 0.020*"offer" + 0.019*"coast" + 0.014*"gold" + 0.013*"plan" + 0.013*"local" + 0.012*"victori" + 0.012*"clear" + 0.012*"year" + 0.010*"black" + 0.009*"power"


Topic: 1 
Words: 0.021*"closer" + 0.015*"protest" + 0.015*"centr" + 0.015*"howard" + 0.013*"busi" + 0.012*"plan" + 0.012*"sale" + 0.011*"famili" + 0.010*"bush" + 0.009*"damag"


Topic: 2 
Words: 0.049*"water" + 0.023*"boost" + 0.022*"miss" + 0.020*"rise" + 0.014*"plan" + 0.013*"fear" + 0.013*"communiti" + 0.012*"servic" + 0.012*"push" + 0.012*"price"


Topic: 3 
Words: 0.025*"nation" + 0.019*"record" + 0.016*"farmer" + 0.016*"market" + 0.015*"drought" + 0.014*"final" + 0.014*"time" + 0.013*"storm" + 0.012*"action" + 0.011*"compani"


Topic: 4 
Words: 0.034*"court" + 0.032*"face" + 0.023*"charg" + 0.016*"murder" + 0.016*"case" + 0.015*"jail" + 0.015*"accus" + 0.013*"nuclear" + 0.013*"trial" + 0.012*"chief"


Topic: 5 
Words: 0.041*"kill" + 0.024*"crash" + 0.018*"iraq" + 0.018*"die" + 0.014*"welcom" + 0.014*"return

## Step 4.2 Running LDA using TF-IDF

In [37]:
'''
Define lda model using corpus_tfidf
'''

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [38]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.009*"timor" + 0.007*"market" + 0.006*"price" + 0.006*"govt" + 0.006*"deal" + 0.006*"hick" + 0.005*"trade" + 0.005*"week" + 0.005*"revamp" + 0.005*"petrol"


Topic: 1 Word: 0.029*"closer" + 0.008*"safeti" + 0.008*"gold" + 0.007*"concern" + 0.007*"drink" + 0.007*"plan" + 0.007*"coast" + 0.006*"council" + 0.006*"worri" + 0.005*"govt"


Topic: 2 Word: 0.014*"water" + 0.012*"govt" + 0.010*"drought" + 0.008*"fund" + 0.007*"urg" + 0.007*"rain" + 0.007*"council" + 0.007*"plan" + 0.007*"farmer" + 0.006*"health"


Topic: 3 Word: 0.007*"burn" + 0.006*"teacher" + 0.006*"centr" + 0.005*"govt" + 0.005*"govern" + 0.005*"lawyer" + 0.005*"anti" + 0.005*"lebanon" + 0.005*"recycl" + 0.004*"say"


Topic: 4 Word: 0.007*"rudd" + 0.007*"cancer" + 0.007*"govt" + 0.006*"doubt" + 0.006*"rise" + 0.006*"whale" + 0.006*"bail" + 0.005*"patient" + 0.005*"export" + 0.005*"violenc"


Topic: 5 Word: 0.020*"polic" + 0.020*"crash" + 0.012*"investig" + 0.012*"die" + 0.010*"charg" + 0.010*"teen" + 0.009*"f

##  Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

In [39]:
'''
Text of sample document 4310
'''

processed_docs[4310]

'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3949936032295227	 
Topic: 0.051*"council" + 0.027*"plan" + 0.016*"elect" + 0.012*"land" + 0.011*"develop" + 0.010*"safeti" + 0.010*"west" + 0.010*"mayor" + 0.009*"fall" + 0.009*"health"

Score: 0.24483314156532288	 
Topic: 0.064*"govt" + 0.032*"urg" + 0.019*"claim" + 0.019*"help" + 0.017*"say" + 0.016*"hous" + 0.015*"reject" + 0.015*"opposit" + 0.014*"worker" + 0.013*"fund"

Score: 0.2201269418001175	 
Topic: 0.049*"water" + 0.023*"boost" + 0.022*"miss" + 0.020*"rise" + 0.014*"plan" + 0.013*"fear" + 0.013*"communiti" + 0.012*"servic" + 0.012*"push" + 0.012*"price"

Score: 0.02001717872917652	 
Topic: 0.021*"closer" + 0.015*"protest" + 0.015*"centr" + 0.015*"howard" + 0.013*"busi" + 0.012*"plan" + 0.012*"sale" + 0.011*"famili" + 0.010*"bush" + 0.009*"damag"

Score: 0.020008007064461708	 
Topic: 0.025*"nation" + 0.019*"record" + 0.016*"farmer" + 0.016*"market" + 0.015*"drought" + 0.014*"final" + 0.014*"time" + 0.013*"storm" + 0.012*"action" + 0.011*"compani"

Score: 0.020004278

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [40]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''

for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5552555918693542	 
Topic: 0.014*"water" + 0.012*"govt" + 0.010*"drought" + 0.008*"fund" + 0.007*"urg" + 0.007*"rain" + 0.007*"council" + 0.007*"plan" + 0.007*"farmer" + 0.006*"health"

Score: 0.28469714522361755	 
Topic: 0.008*"death" + 0.008*"polic" + 0.008*"bodi" + 0.007*"fire" + 0.006*"solomon" + 0.006*"toll" + 0.005*"quit" + 0.005*"murder" + 0.005*"inquest" + 0.004*"court"

Score: 0.020006943494081497	 
Topic: 0.009*"firefight" + 0.007*"blaze" + 0.007*"elect" + 0.005*"uranium" + 0.005*"liber" + 0.005*"lead" + 0.005*"south" + 0.005*"tiger" + 0.005*"bird" + 0.005*"east"

Score: 0.020006675273180008	 
Topic: 0.007*"rudd" + 0.007*"cancer" + 0.007*"govt" + 0.006*"doubt" + 0.006*"rise" + 0.006*"whale" + 0.006*"bail" + 0.005*"patient" + 0.005*"export" + 0.005*"violenc"

Score: 0.020006097853183746	 
Topic: 0.029*"closer" + 0.008*"safeti" + 0.008*"gold" + 0.007*"concern" + 0.007*"drink" + 0.007*"plan" + 0.007*"coast" + 0.006*"council" + 0.006*"worri" + 0.005*"govt"

Score: 0.0200

# Step 6: Testing model on unseen document¶

In [41]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6200627684593201	 Topic: 0.025*"nation" + 0.019*"record" + 0.016*"farmer" + 0.016*"market" + 0.015*"drought"
Score: 0.21990956366062164	 Topic: 0.021*"closer" + 0.015*"protest" + 0.015*"centr" + 0.015*"howard" + 0.013*"busi"
Score: 0.020004117861390114	 Topic: 0.021*"test" + 0.020*"open" + 0.020*"lead" + 0.018*"world" + 0.017*"win"
Score: 0.020003437995910645	 Topic: 0.051*"council" + 0.027*"plan" + 0.016*"elect" + 0.012*"land" + 0.011*"develop"
Score: 0.02000335603952408	 Topic: 0.020*"offer" + 0.019*"coast" + 0.014*"gold" + 0.013*"plan" + 0.013*"local"
Score: 0.02000335603952408	 Topic: 0.034*"court" + 0.032*"face" + 0.023*"charg" + 0.016*"murder" + 0.016*"case"
Score: 0.02000335417687893	 Topic: 0.049*"water" + 0.023*"boost" + 0.022*"miss" + 0.020*"rise" + 0.014*"plan"
Score: 0.02000335417687893	 Topic: 0.041*"kill" + 0.024*"crash" + 0.018*"iraq" + 0.018*"die" + 0.014*"welcom"
Score: 0.02000335417687893	 Topic: 0.064*"govt" + 0.032*"urg" + 0.019*"claim" + 0.019*"help" + 0.0